# Melbourne Electronics Store

## Relational Model

**Libraries and imports**

In [1]:
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from faker import Faker

# from warnings import simplefilter
# simplefilter('ignore')
pd.set_option('display.max_columns', None)

## Load `original_df` into Jupyter Notebook

In [2]:
original_df = pd.read_csv('../data/raw/original_data.csv')
original_df.head()

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD493666,ID0844489306,01 27 2019,Thompson,"[('Lucent 330S', 1), ('pearTV', 2), ('iAssist ...",18300.0,74.75,-37.820755,144.948063,0,18374.75,Summer,False,0.9039,love it received the lucent g very fast and in...,True
1,ORD129378,ID6167417934,02 03 2019,Nickolson,"[('Olivia x460', 1), ('Universe Note', 2)]",8125.0,80.74,-37.814972,144.960240,10,7393.24,Autumn,True,0.9127,nice battery life great phone overall,True
2,ORD455246,ID4326586172,02-24-2019,Thompson,"[('iAssist Line', 1), ('Alcon 10', 2), ('Candl...",20985.0,81.66,-37.800770,144.957410,15,17918.91,Summer,False,1.6071,five stars absolutely fabulous,True
3,ORD497096,ID4735909071,03 10 2019,Thompson,"[('pearTV', 2), ('Thunder line', 2)]",16980.0,103.77,-37.804318,144.950049,5,16234.77,Spring,True,0.9663,this phone is wonderful!! olivia really out di...,True
4,ORD414419,ID0207085738,03 29 2019,Bakers,"[('iStream', 1), ('pearTV', 1)]",6460.0,81.31,-37.812585,145.015529,5,6218.31,Autumn,True,1.8081,awesome! the product fit the description. i lo...,True


In [3]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       1000 non-null   object 
 1   customer_id                    1000 non-null   object 
 2   date                           1000 non-null   object 
 3   nearest_warehouse              990 non-null    object 
 4   shopping_cart                  1000 non-null   object 
 5   order_price                    990 non-null    float64
 6   delivery_charges               1000 non-null   float64
 7   customer_lat                   990 non-null    float64
 8   customer_long                  990 non-null    float64
 9   coupon_discount                1000 non-null   int64  
 10  order_total                    990 non-null    float64
 11  season                         990 non-null    object 
 12  is_expedited_delivery          1000 non-null   bo

The dataset was converted to csv in our pre-EDA analysis, and is in good shape from the previous transormations.

From it, we can generate the following tables:
- customers
- products
- orders
- order_items; and
- reviews (if we extrapolate a little bit)

And we already have the following tables:
- warehouses

Let's start creating the customers table, then:

## Creating Entities (tables)

### Warehouses

In [47]:
warehouses_df = pd.read_csv('../data/raw/warehouses.csv')
warehouses_df

,names,lat,lon
0,Nickolson,-37.818595,144.969551
1,Thompson,-37.812673,144.947069
2,Bakers,-37.809996,144.995232


In [54]:
warehouses_df['warehouse_id'] = ['MELBNICK', 'MELBTHOM', 'MELBBAKE']

In [55]:
warehouses_df = warehouses_df.rename({
    'names': 'warehouse_name',
    'lat': 'lat',
    'lon': 'lon',
    'warehouse_id': 'warehouse_id'
}, axis = 1)

warehouses_df = warehouses_df[['warehouse_id', 'warehouse_name', 'lat', 'lon']]
warehouses_df

,warehouse_id,warehouse_name,lat,lon
0,MELBNICK,Nickolson,-37.818595,144.969551
1,MELBTHOM,Thompson,-37.812673,144.947069
2,MELBBAKE,Bakers,-37.809996,144.995232


In [62]:
warehouses_df.to_csv('../data/relational/warehouses.csv', index = False)

! ls ../data/relational/

customers.csv     order_reviews.csv products.csv
order_items.csv   orders.csv        warehouses.csv


## Original dataset

In [63]:
warehouse_id_name_dict = dict(zip(warehouses_df['warehouse_name'], warehouses_df['warehouse_id']))
warehouse_id_name_dict

{'Nickolson': 'MELBNICK', 'Thompson': 'MELBTHOM', 'Bakers': 'MELBBAKE'}

In [64]:
display(original_df.head())
original_df.shape

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer,nearest_warehouse_id
0,ORD493666,ID0844489306,01 27 2019,Thompson,"[('Lucent 330S', 1), ('pearTV', 2), ('iAssist ...",18300.0,74.75,-37.820755,144.948063,0,18374.75,Summer,False,0.9039,love it received the lucent g very fast and in...,True,MELBTHOM
1,ORD129378,ID6167417934,02 03 2019,Nickolson,"[('Olivia x460', 1), ('Universe Note', 2)]",8125.0,80.74,-37.814972,144.960240,10,7393.24,Autumn,True,0.9127,nice battery life great phone overall,True,MELBNICK
2,ORD455246,ID4326586172,02-24-2019,Thompson,"[('iAssist Line', 1), ('Alcon 10', 2), ('Candl...",20985.0,81.66,-37.800770,144.957410,15,17918.91,Summer,False,1.6071,five stars absolutely fabulous,True,MELBTHOM
3,ORD497096,ID4735909071,03 10 2019,Thompson,"[('pearTV', 2), ('Thunder line', 2)]",16980.0,103.77,-37.804318,144.950049,5,16234.77,Spring,True,0.9663,this phone is wonderful!! olivia really out di...,True,MELBTHOM
4,ORD414419,ID0207085738,03 29 2019,Bakers,"[('iStream', 1), ('pearTV', 1)]",6460.0,81.31,-37.812585,145.015529,5,6218.31,Autumn,True,1.8081,awesome! the product fit the description. i lo...,True,MELBBAKE


(1000, 17)

In [65]:
original_df['nearest_warehouse_id'] = original_df['nearest_warehouse'].map(warehouse_id_name_dict)

### Customers

In [66]:
# How many unique customers?

original_df['customer_id'].nunique()

973

In [67]:
customers_df = original_df.drop_duplicates(subset=['customer_id'], keep='first')[['customer_id', 
                                                                                  'customer_lat', 
                                                                                  'customer_long', 
                                                                                  'nearest_warehouse_id',
                                                                                  'is_happy_customer']]

customers_df.head()

,customer_id,customer_lat,customer_long,nearest_warehouse_id,is_happy_customer
0,ID0844489306,-37.820755,144.948063,MELBTHOM,True
1,ID6167417934,-37.814972,144.960240,MELBNICK,True
2,ID4326586172,-37.800770,144.957410,MELBTHOM,True
3,ID4735909071,-37.804318,144.950049,MELBTHOM,True
4,ID0207085738,-37.812585,145.015529,MELBBAKE,True


In [68]:
customers_df.to_csv('../data/relational/customers.csv', index=False)

! ls ../data/relational

customers.csv     order_reviews.csv products.csv
order_items.csv   orders.csv        warehouses.csv


### Products

In [69]:
# Save regex pattern for product names in a variable

product_pattern = (r'\'(\w* ?\d*[\w.]*)\'')

regex_results_df = original_df['shopping_cart'].str.extractall(product_pattern, flags=re.IGNORECASE).reset_index()
regex_results_df[0] = regex_results_df[0].str.title()
regex_results_df

,level_0,match,0
0,0,0,Lucent 330S
1,0,1,Peartv
2,0,2,Iassist Line
3,1,0,Olivia X460
4,1,1,Universe Note
...,...,...,...
2986,998,1,Thunder Line
2987,998,2,Istream
2988,999,0,Thunder Line
2989,999,1,Istream


We have the list of products from the `original_df`.

Now if we rearagne a little, we can have the `products_df`.

Let's do it:

In [70]:
products_set = set(regex_results_df.loc[:, 0])
products_set

{'Alcon 10',
 'Candle Inferno',
 'Iassist Line',
 'Istream',
 'Lucent 330S',
 'Olivia X460',
 'Peartv',
 'Thunder Line',
 'Toshika 750',
 'Universe Note'}

In [71]:
# Create Products entity

pre_products_df = pd.DataFrame(products_set, columns=['product_name'])
pre_products_df

,product_name
0,Peartv
1,Iassist Line
2,Candle Inferno
3,Olivia X460
4,Alcon 10
5,Toshika 750
6,Universe Note
7,Istream
8,Thunder Line
9,Lucent 330S


In [72]:
# Create product_id from Faker

faker = Faker()

pre_products_df['product_id'] = [faker.ean13() for i in range (len(products_set))]
pre_products_df

,product_name,product_id
0,Peartv,9282452748957
1,Iassist Line,4935004868836
2,Candle Inferno,8841305784439
3,Olivia X460,4771123833320
4,Alcon 10,9624222870145
5,Toshika 750,6373198616167
6,Universe Note,0488066261311
7,Istream,0124625445524
8,Thunder Line,4076942951638
9,Lucent 330S,5827539463497


In [73]:
# Organize features in prep work for Database
products_df = pre_products_df[['product_id', 'product_name']]
products_df

,product_id,product_name
0,9282452748957,Peartv
1,4935004868836,Iassist Line
2,8841305784439,Candle Inferno
3,4771123833320,Olivia X460
4,9624222870145,Alcon 10
5,6373198616167,Toshika 750
6,0488066261311,Universe Note
7,0124625445524,Istream
8,4076942951638,Thunder Line
9,5827539463497,Lucent 330S


In [74]:
products_df.to_csv('../data/relational/products.csv', index=False)

! ls ../data/relational/

customers.csv     order_reviews.csv products.csv
order_items.csv   orders.csv        warehouses.csv


### Order Items

In [75]:
original_df.reset_index()[['index', 'order_id', 'customer_id']]

,index,order_id,customer_id
0,0,ORD493666,ID0844489306
1,1,ORD129378,ID6167417934
2,2,ORD455246,ID4326586172
3,3,ORD497096,ID4735909071
4,4,ORD414419,ID0207085738
...,...,...,...
995,995,ORD182419,ID0595857779
996,996,ORD116440,ID0332546017
997,997,ORD229269,ID0582156569
998,998,ORD032042,ID1725216340


In [76]:
pre_order_items_df = original_df.reset_index()[['index', 'order_id', 'customer_id']]
pre_order_items_df.head()

,index,order_id,customer_id
0,0,ORD493666,ID0844489306
1,1,ORD129378,ID6167417934
2,2,ORD455246,ID4326586172
3,3,ORD497096,ID4735909071
4,4,ORD414419,ID0207085738


In [77]:
regex_results_df

,level_0,match,0
0,0,0,Lucent 330S
1,0,1,Peartv
2,0,2,Iassist Line
3,1,0,Olivia X460
4,1,1,Universe Note
...,...,...,...
2986,998,1,Thunder Line
2987,998,2,Istream
2988,999,0,Thunder Line
2989,999,1,Istream


In [78]:
pre_order_items_df_merged = pre_order_items_df.merge(regex_results_df, left_on='index', right_on='level_0')
pre_order_items_df_merged.head()

,index,order_id,customer_id,level_0,match,0
0,0,ORD493666,ID0844489306,0,0,Lucent 330S
1,0,ORD493666,ID0844489306,0,1,Peartv
2,0,ORD493666,ID0844489306,0,2,Iassist Line
3,1,ORD129378,ID6167417934,1,0,Olivia X460
4,1,ORD129378,ID6167417934,1,1,Universe Note


In [79]:
pre_order_items_df_merged.columns = ['index', 'order_id', 'customer_id', 'level_0', 'match', 'product_name']
pre_order_items_df_merged.head()

,index,order_id,customer_id,level_0,match,product_name
0,0,ORD493666,ID0844489306,0,0,Lucent 330S
1,0,ORD493666,ID0844489306,0,1,Peartv
2,0,ORD493666,ID0844489306,0,2,Iassist Line
3,1,ORD129378,ID6167417934,1,0,Olivia X460
4,1,ORD129378,ID6167417934,1,1,Universe Note


In [80]:
product_id_name_dict = dict(zip(products_df['product_name'], products_df['product_id']))
product_id_name_dict

{'Peartv': '9282452748957',
 'Iassist Line': '4935004868836',
 'Candle Inferno': '8841305784439',
 'Olivia X460': '4771123833320',
 'Alcon 10': '9624222870145',
 'Toshika 750': '6373198616167',
 'Universe Note': '0488066261311',
 'Istream': '0124625445524',
 'Thunder Line': '4076942951638',
 'Lucent 330S': '5827539463497'}

In [81]:
pre_order_items_df_merged['product_id'] = pre_order_items_df_merged['product_name'].map(product_id_name_dict)
pre_order_items_df_merged

,index,order_id,customer_id,level_0,match,product_name,product_id
0,0,ORD493666,ID0844489306,0,0,Lucent 330S,5827539463497
1,0,ORD493666,ID0844489306,0,1,Peartv,9282452748957
2,0,ORD493666,ID0844489306,0,2,Iassist Line,4935004868836
3,1,ORD129378,ID6167417934,1,0,Olivia X460,4771123833320
4,1,ORD129378,ID6167417934,1,1,Universe Note,0488066261311
...,...,...,...,...,...,...,...
2986,998,ORD032042,ID1725216340,998,1,Thunder Line,4076942951638
2987,998,ORD032042,ID1725216340,998,2,Istream,0124625445524
2988,999,ORD227618,ID0571730335,999,0,Thunder Line,4076942951638
2989,999,ORD227618,ID0571730335,999,1,Istream,0124625445524


In [82]:
len(np.arange(len(pre_order_items_df_merged)) + 1)

2991

In [83]:
pre_order_items_df_merged['order_item_id'] = np.arange(len(pre_order_items_df_merged)) + 1
pre_order_items_df_merged.head()

,index,order_id,customer_id,level_0,match,product_name,product_id,order_item_id
0,0,ORD493666,ID0844489306,0,0,Lucent 330S,5827539463497,1
1,0,ORD493666,ID0844489306,0,1,Peartv,9282452748957,2
2,0,ORD493666,ID0844489306,0,2,Iassist Line,4935004868836,3
3,1,ORD129378,ID6167417934,1,0,Olivia X460,4771123833320,4
4,1,ORD129378,ID6167417934,1,1,Universe Note,0488066261311,5


In [84]:
pre_order_items_df_org = pre_order_items_df_merged[['order_item_id', 
                                                    'order_id', 
                                                    'customer_id', 
                                                    'product_id', 
                                                    'product_name', 'level_0', 'match']]

pre_order_items_df_org['temp_key'] = pre_order_items_df_org.loc[:, 'level_0'].astype(str) + \
                                                        '_' + pre_order_items_df_org.loc[:, 'match'].astype(str)

pre_order_items_df_org.head()

,order_item_id,order_id,customer_id,product_id,product_name,level_0,match,temp_key
0,1,ORD493666,ID0844489306,5827539463497,Lucent 330S,0,0,0_0
1,2,ORD493666,ID0844489306,9282452748957,Peartv,0,1,0_1
2,3,ORD493666,ID0844489306,4935004868836,Iassist Line,0,2,0_2
3,4,ORD129378,ID6167417934,4771123833320,Olivia X460,1,0,1_0
4,5,ORD129378,ID6167417934,0488066261311,Universe Note,1,1,1_1


In [85]:
# Save regex pattern for product quantities in a variable
quantity_pattern = r"(\d{1})\)"

qty_regex_results_df = original_df['shopping_cart'].str.extractall(quantity_pattern, flags=re.IGNORECASE).reset_index()
qty_regex_results_df.columns = ['level_0', 'match', 'product_quantity']
qty_regex_results_df['temp_key'] = qty_regex_results_df.loc[:, 'level_0'].astype(str) + \
                                                        '_' + qty_regex_results_df.loc[:, 'match'].astype(str)

qty_regex_results_df.head()

,level_0,match,product_quantity,temp_key
0,0,0,1,0_0
1,0,1,2,0_1
2,0,2,2,0_2
3,1,0,1,1_0
4,1,1,2,1_1


In [86]:
order_items_df = pre_order_items_df_org\
    .merge(qty_regex_results_df[['temp_key', 'product_quantity']], on = 'temp_key')[['order_item_id', 
                                                                                     'order_id', 
                                                                                     'customer_id', 
                                                                                     'product_id',
#                                                                                      'product_name',
                                                                                     'product_quantity']]

order_items_df.head()

,order_item_id,order_id,customer_id,product_id,product_quantity
0,1,ORD493666,ID0844489306,5827539463497,1
1,2,ORD493666,ID0844489306,9282452748957,2
2,3,ORD493666,ID0844489306,4935004868836,2
3,4,ORD129378,ID6167417934,4771123833320,1
4,5,ORD129378,ID6167417934,0488066261311,2


From it, we can generate the following tables:
- ~~customers~~
- ~~products~~
- orders
- ~~order_items~~; and
- order_reviews (if we extrapolate a little bit)
- deliveries (maybe)

In [87]:
order_items_df.to_csv('../data/relational/order_items.csv', index=False)

! ls ../data/relational/

customers.csv     order_reviews.csv products.csv
order_items.csv   orders.csv        warehouses.csv


### Orders

In [88]:
original_df.columns.to_list()

['order_id',
 'customer_id',
 'date',
 'nearest_warehouse',
 'shopping_cart',
 'order_price',
 'delivery_charges',
 'customer_lat',
 'customer_long',
 'coupon_discount',
 'order_total',
 'season',
 'is_expedited_delivery',
 'distance_to_nearest_warehouse',
 'latest_customer_review',
 'is_happy_customer',
 'nearest_warehouse_id']

In [90]:
orders_df = original_df[
    [
        'order_id',
        'customer_id',
        'date',
        'is_expedited_delivery',
        'nearest_warehouse_id',
        'distance_to_nearest_warehouse',
        'order_price',
        'delivery_charges',
        'coupon_discount',
        'order_total',
    ]
]

orders_df.head()

,order_id,customer_id,date,is_expedited_delivery,nearest_warehouse_id,distance_to_nearest_warehouse,order_price,delivery_charges,coupon_discount,order_total
0,ORD493666,ID0844489306,01 27 2019,False,MELBTHOM,0.9039,18300.0,74.75,0,18374.75
1,ORD129378,ID6167417934,02 03 2019,True,MELBNICK,0.9127,8125.0,80.74,10,7393.24
2,ORD455246,ID4326586172,02-24-2019,False,MELBTHOM,1.6071,20985.0,81.66,15,17918.91
3,ORD497096,ID4735909071,03 10 2019,True,MELBTHOM,0.9663,16980.0,103.77,5,16234.77
4,ORD414419,ID0207085738,03 29 2019,True,MELBBAKE,1.8081,6460.0,81.31,5,6218.31


In [91]:
orders_df.to_csv('../data/relational/orders.csv', index=False)

!ls ../data/relational/

customers.csv     order_reviews.csv products.csv
order_items.csv   orders.csv        warehouses.csv


### Order Reviews

In [92]:
order_reviews_df = original_df[['order_id', 'customer_id', 'latest_customer_review']]

order_reviews_df.head()

,order_id,customer_id,latest_customer_review
0,ORD493666,ID0844489306,love it received the lucent g very fast and in...
1,ORD129378,ID6167417934,nice battery life great phone overall
2,ORD455246,ID4326586172,five stars absolutely fabulous
3,ORD497096,ID4735909071,this phone is wonderful!! olivia really out di...
4,ORD414419,ID0207085738,awesome! the product fit the description. i lo...


In [93]:
order_reviews_df.to_csv('../data/relational/order_reviews.csv', index = False)

!ls ../data/relational/

customers.csv     order_reviews.csv products.csv
order_items.csv   orders.csv        warehouses.csv
